In [31]:
from functools import total_ordering
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time
import os
import numpy as np


In [3]:
user_zip = input('Enter a zip code and press enter ')
print(f'Searching for Utilities serving  {user_zip} and the surrrounding cities for your county')

Enter a zip code and press enter 97045
Searching for Utilities serving  97045 and the surrrounding cities for your county


In [5]:
page="https://www.ewg.org/tapwater/search-results.php?zip5="+ str(user_zip)+"&searchtype=zip"
url = requests.get(page)
table = pd.read_html(url.text)

In [6]:
table[0]

,Utility name,City,People served
0,North Clackamas County Water Commission,"Oregon City, OR","Population served:87,700"
1,South Fork Water Board,"Oregon City, OR","Population served:65,000"
2,Sunrise Water Authority,"Happy Valley, OR","Population served:46,228"
3,Clackamas River Water - Clackamas,"Clackamas, OR","Population served:41,338"
4,Lake Oswego Municipal Water,"Lake Oswego, OR","Population served:36,453"
5,Oregon City,"Oregon City, OR","Population served:33,940"
6,Oak Lodge Water District,"Milwaukie, OR","Population served:30,000"
7,City of West Linn,"West Linn, OR","Population served:25,002"
8,City of Milwaukie,"Milwaukie, OR","Population served:20,500"
9,City of Wilsonville,"Wilsonville, OR","Population served:22,729"


In [7]:
table[1]

,Utility name,City,People served
0,Alder Creek Barlow Water District,Sandy,Population served: 310
1,Ashdown Wood Water Company,Newberg,Population served: 75
2,Barlow Trail Estates,Portland,Population served: 180
3,"Barlow, City of",Canby,Population served: 140
4,Big Foot Mobile Park,Sandy,Population served: 60
...,...,...,...
76,Wildwood Annex Water District,Welches,Population served: 35
77,Workman Air Park #3,Newberg,Population served: 75
78,Wunder Mobile Park,Molalla,Population served: 100
79,Zig Zag Village Water System,Molalla,Population served: 80


In [8]:
utilities_df = pd.concat([table[0], table[1]], ignore_index=True)

In [9]:
utilities_df

,Utility name,City,People served
0,Alder Creek Barlow Water District,Sandy,Population served: 310
1,Ashdown Wood Water Company,Newberg,Population served: 75
2,Barlow Trail Estates,Portland,Population served: 180
3,"Barlow, City of",Canby,Population served: 140
4,Big Foot Mobile Park,Sandy,Population served: 60
...,...,...,...
86,Oregon City,"Oregon City, OR","Population served:33,940"
87,Oak Lodge Water District,"Milwaukie, OR","Population served:30,000"
88,City of West Linn,"West Linn, OR","Population served:25,002"
89,City of Milwaukie,"Milwaukie, OR","Population served:20,500"


In [10]:
#Build a list of the utilities to visit and scrape data from EWG
utility_list = utilities_df['Utility name'].to_list()
utility_list

['Alder Creek Barlow Water District',
 'Ashdown Wood Water Company',
 'Barlow Trail Estates',
 'Barlow, City of',
 'Big Foot Mobile Park',
 'Big Valley Woods Water Corporation',
 'Boring Water District No. 24',
 'Brightwood Water Works',
 'Canby Regency',
 'Canby Utility',
 'Carver Mobile Ranch',
 'Cedarhurst Improvement Club',
 'City of Gladstone',
 'City of Milwaukie',
 'City of West Linn',
 'City of Wilsonville',
 'Clackamas River Water - Clackamas',
 'Clackamas River Water - Clairmont',
 'Colton Water District',
 'Country Club Water District',
 'Currinsville Mobile Home Park',
 'Dietz Air Park Water System',
 'Eagle Creek Mobile Estates',
 'Eagle Crest Mobile Home Park',
 'Eastmont Water Company',
 'Estacada Mobile Village',
 'Estacada, City of',
 'Excalibur Village',
 'Forest Haven Subdivision',
 'Forest Park Mobile Village',
 'Glenmorrie Cooperative Association',
 'Gnr Water District',
 'Government Camp Water System',
 'Hiland Water Company - Shadow Wood',
 'Kelso Water Associati

In [11]:
start_time = time.time()

#Begin Scrape
driver = webdriver.Chrome()

contaminant_list = []
for utility in utility_list:
    try:
        driver.get("https://www.ewg.org/tapwater/advanced-search.php")
        time.sleep(1)
        utility_input = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[1]')
        utility_input.clear()
        utility_input.send_keys(utility)
        go_btn = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[3]')
        go_btn.click()
        time.sleep(1)
        utility = driver.find_element(By.XPATH, '/html/body/div[3]/main/figure/table/tbody/tr/td[1]/a')
        utility.click()

        page_source = driver.page_source

        html_soup = soup(page_source, 'html.parser')

        #Get the name of the Water Utility
        Utility = html_soup.find('h1').text
        Utility


        #get the html data we need
        data_box = html_soup.find_all('div', class_='contaminant-name')

        for i in range(len(data_box)):
            data = data_box[i].find_all('span')
            data_measure = []
            d = {
                'Utility' : Utility,
                'Contaminant': '', 
                'Utility Measuremnt':'', 
                'EWG HEALTH GUIDELINE': '',
                'Legal Limit':'' 
            }

            contaminant_name = data_box[i].find('h3')
            d['Contaminant'] = contaminant_name

            for j in range(len(data)):
                measurement = data[j].text
                #print(measurement)
                data_measure.append(measurement)
                #print(data_measure)

            try:
                d['Utility Measuremnt'] = data_measure[data_measure.index('THIS UTILITY')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong")
            try:
                d['EWG HEALTH GUIDELINE'] = data_measure[data_measure.index('EWG HEALTH GUIDELINE')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong") 
            try:
                d['Legal Limit'] = data_measure[data_measure.index('LEGAL LIMIT')+1]
            except ValueError:
                print("A value error arose")
            except:
                print("Something else went wrong") 

            contaminant_list.append(d)
    except:
        pass
print ('Done Scraping, moving on to dataset cleaning and file writing')
#Construct a dataframe from the results
scraped_df = pd.DataFrame(contaminant_list)

#Define a function to strip the h3 tages from the contaminants (relict from scraping)
import re
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

scraped_df = scraped_df.astype({"Contaminant": str})

scraped_df.Contaminant = scraped_df.Contaminant.apply(lambda x: cleanhtml(x))

# #Output the resulting dataframe to a csv file
# state_path_contaminant_output = os.path.join(path,'contaminants.csv')
# scraped_df.to_csv(state_path_contaminant_output, index=False)

finish_time = time.time()

total_time = (finish_time - start_time)/60

print(f'The process finished in finished in {total_time} minutes')

A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose


A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose


In [12]:
scraped_df

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,Alder Creek Barlow Water District,Haloacetic acids (HAA5)†,30.8 ppb,0.1 ppb,60 ppb
1,Alder Creek Barlow Water District,Total trihalomethanes (TTHMs)†,38.3 ppb,0.15 ppb,80 ppb
2,Alder Creek Barlow Water District,Barium*,1.47 ppb,700 ppb,"2,000 ppb"
3,Alder Creek Barlow Water District,Fluoride*,0.0375 ppm,,4 ppm
4,Alder Creek Barlow Water District,Nitrate*,0.0566 ppm,0.14 ppm,10 ppm
...,...,...,...,...,...
683,City of Wilsonville,Chromium (total),0.00588 ppb,,100 ppb
684,City of Wilsonville,Manganese,3.12 ppb,100 ppb,
685,City of Wilsonville,Strontium,0.0380 ppb,"1,500 ppb",
686,City of Wilsonville,Vanadium,1.59 ppb,21 ppb,


In [37]:
df = scraped_df.copy()
df.head()

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,Alder Creek Barlow Water District,Haloacetic acids (HAA5)†,30.8 ppb,0.1 ppb,60 ppb
1,Alder Creek Barlow Water District,Total trihalomethanes (TTHMs)†,38.3 ppb,0.15 ppb,80 ppb
2,Alder Creek Barlow Water District,Barium*,1.47 ppb,700 ppb,"2,000 ppb"
3,Alder Creek Barlow Water District,Fluoride*,0.0375 ppm,,4 ppm
4,Alder Creek Barlow Water District,Nitrate*,0.0566 ppm,0.14 ppm,10 ppm


In [38]:
#Drop rows with non EWG Guideline for a contaminant
#df3[df3['Contaminant'].notna()]
#df3['EWG HEALTH GUIDELINE'].notnull().value_counts()
df =df.replace(r'^\s*$', np.nan, regex=True)

In [39]:
df.sample(30)

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
542,Welches Water Company,Barium,4.00 ppb,700 ppb,"2,000 ppb"
493,South Fork Water Board,None,NaN,NaN,NaN
529,Totem Village Mobile Home Park,Barium,7.53 ppb,700 ppb,"2,000 ppb"
637,Oak Lodge Water District,Chlorate,44.0 ppb,210 ppb,NaN
9,Ashdown Wood Water Company,None,NaN,NaN,NaN
475,Sleepy Hollow Water Company,Dibromoacetic acid,2.63 ppb,0.04 ppb,NaN
602,Lake Oswego Municipal Water,Strontium,0.0333 ppb,"1,500 ppb",NaN
540,Welches Water Company,Nitrate,0.734 ppm,0.14 ppm,10 ppm
230,Gnr Water District,Nitrate*,0.111 ppm,0.14 ppm,10 ppm
261,Lake Grove Water Association,Dichloroacetic acid,3.33 ppb,0.2 ppb,NaN


In [40]:
df = df.dropna()

In [41]:
df.sample(20)

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
568,South Fork Water Board,Barium,2.89 ppb,700 ppb,"2,000 ppb"
59,Cedarhurst Improvement Club,Arsenic,4.00 ppb,0.004 ppb,10 ppb
449,Salmon Valley Water Company,Arsenic,0.400 ppb,0.004 ppb,10 ppb
214,Glenmorrie Cooperative Association,Barium,3.15 ppb,700 ppb,"2,000 ppb"
69,City of Gladstone,Barium*,1.77 ppb,700 ppb,"2,000 ppb"
461,Skylands Water Company,Nitrate and nitrite,0.302 ppm,0.14 ppm,10 ppm
556,Zig Zag Water Co-op,Haloacetic acids (HAA5)†,0.143 ppb,0.1 ppb,60 ppb
46,Canby Utility,Atrazine,0.0189 ppb,0.1 ppb,3 ppb
295,Mossy Brae Water District,Barium,7.98 ppb,700 ppb,"2,000 ppb"
316,North Brightwood Improv Association,Arsenic,3.40 ppb,0.004 ppb,10 ppb


In [43]:
df3 = df.copy()
#Split of the contaminant measurements and units and people served
df3['Units'] = df3['Utility Measuremnt'].apply(lambda x: x.split()[-1])
df3['Utility Measuremnt'] = df3['Utility Measuremnt'].apply(lambda x: x.split()[0])
df3['EWG HEALTH GUIDELINE'] = df3['EWG HEALTH GUIDELINE'].apply(lambda x: x.split()[0])
df3['Legal Limit']=df3['Legal Limit'].apply(lambda x: '0 units' if pd.isnull(x) else x)
df3['Legal Limit'] = df3['Legal Limit'].apply(lambda x: x.split()[0])
#df3['People served'] = df3['People served'].apply(lambda x: x.split(':')[-1])

In [44]:
#Replace any commas in the thousandths place
df3.replace(',','', regex=True, inplace=True)

In [45]:
#Change the datatype of the measurements and EWG Guidlines to numeric values
df3['Utility Measuremnt'] = pd.to_numeric(df3['Utility Measuremnt'])
df3['EWG HEALTH GUIDELINE'] = pd.to_numeric(df3['EWG HEALTH GUIDELINE'])
#df3['People served'] = pd.to_numeric(df3['People served'])

In [46]:
#Define the Contaminant Factor (how many times larger is the utility measurement than EWG guideline)
df3['Contaminant_Factor'] = df3['Utility Measuremnt']/df3['EWG HEALTH GUIDELINE']

In [48]:
df3

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit,Units,Contaminant_Factor
0,Alder Creek Barlow Water District,Haloacetic acids (HAA5)†,30.8000,0.10,60,ppb,308.000000
1,Alder Creek Barlow Water District,Total trihalomethanes (TTHMs)†,38.3000,0.15,80,ppb,255.333333
2,Alder Creek Barlow Water District,Barium*,1.4700,700.00,2000,ppb,0.002100
4,Alder Creek Barlow Water District,Nitrate*,0.0566,0.14,10,ppm,0.404286
5,Alder Creek Barlow Water District,Nitrate and nitrite*,0.0665,0.14,10,ppm,0.475000
...,...,...,...,...,...,...,...
676,City of Wilsonville,Haloacetic acids (HAA5)†,5.0200,0.10,60,ppb,50.200000
678,City of Wilsonville,Nitrate,0.4990,0.14,10,ppm,3.564286
679,City of Wilsonville,Nitrate and nitrite,0.4990,0.14,10,ppm,3.564286
680,City of Wilsonville,Total trihalomethanes (TTHMs)†,13.0000,0.15,80,ppb,86.666667


In [62]:
utilities_grouped_df1 = df3.groupby('Utility').count()['Contaminant_Factor']
utilities_grouped_df2 = df3.groupby('Utility').sum()['Contaminant_Factor']


In [63]:
utilities_grouped_df1

Utility
Alder Creek Barlow Water District    5
Ashdown Wood Water Company           1
Barlow City of                       1
Barlow Trail Estates                 2
Big Foot Mobile Park                 3
                                    ..
Wildwood Annex Water District        1
Workman Air Park #3                  1
Wunder Mobile Park                   2
Zig Zag Village Water System         2
Zig Zag Water Co-op                  3
Name: Contaminant_Factor, Length: 78, dtype: int64

In [64]:
utilities_grouped_df2

Utility
Alder Creek Barlow Water District    564.214719
Ashdown Wood Water Company             0.004257
Barlow City of                         7.714286
Barlow Trail Estates                 150.010571
Big Foot Mobile Park                  12.492000
                                        ...    
Wildwood Annex Water District          0.001857
Workman Air Park #3                    0.641429
Wunder Mobile Park                     0.316238
Zig Zag Village Water System           7.504714
Zig Zag Water Co-op                    9.100538
Name: Contaminant_Factor, Length: 78, dtype: float64

In [65]:
Sum_ContaminantFactor = utilities_grouped_df2.sum()
Avg_Contaminant_Factor = utilities_grouped_df2.mean()
Num_Contaminants = utilities_grouped_df1.sum()

In [75]:
#Output to be joined with census data for racial, ethnic, and income indices
data = {
    'Zip' : user_zip,
    'Sum_ContaminantFactor': Sum_ContaminantFactor, 
    'Avg_Contaminant_Factor':Avg_Contaminant_Factor, 
    'Num_Contaminants' : Num_Contaminants
}

df_final = pd.DataFrame(data, index=[0])

In [76]:
df_final

,Zip,Sum_ContaminantFactor,Avg_Contaminant_Factor,Num_Contaminants
0,97045,26970.909776,345.780895,331
